<a href="https://colab.research.google.com/github/chrismoroney/TensorFlow-practice/blob/main/sarcasm_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import json

def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

data = list(parse_data('/tmp/sarcasm.json'))

In [ ]:
from bs4 import BeautifulSoup
import string


table = str.maketrans('', '', string.punctuation)

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

In [ ]:
sentences = []
labels = []
urls = []
for item in data:
  sentence = item['headline'].lower()
  sentence = sentence.replace(",", " , ")
  sentence = sentence.replace(".", " . ")
  sentence = sentence.replace("-", " - ")
  sentence = sentence.replace("/", " / ")
  soup = BeautifulSoup(sentence)
  sentence = soup.get_text()
  words = sentence.split()
  filtered_sentence = ""
  for word in words:
    word = word.translate(table)
    if word not in stopwords:
      filtered_sentence += word + " "
  sentences.append(filtered_sentence)
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

<ipython-input-26-4b289e6b071a>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


In [ ]:
print(len(sentences))

26709


<h1> Split data to training and testing sets </h1>

In [ ]:
training_size = 22000

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [ ]:
vocab_size = 25000
max_length = 25
trunc_type = 'post'
padding_type = 'post'
oov_token = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)

In [ ]:
word_idx = tokenizer.word_index

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
training_padded = pad_sequences(training_sequences, padding = 'post')
testing_padded = pad_sequences(testing_sequences, padding = 'post')
print(word_idx)

{'<OOV>': 1, 'new': 2, 'trump': 3, 'man': 4, 'not': 5, 'just': 6, 'will': 7, 'one': 8, 'year': 9, 'report': 10, 'area': 11, 'donald': 12, 'u': 13, 'day': 14, 'says': 15, 's': 16, 'can': 17, 'woman': 18, 'first': 19, 'time': 20, 'no': 21, 'get': 22, 'like': 23, 'old': 24, 'trumps': 25, 'life': 26, 'now': 27, 'off': 28, 'obama': 29, 'people': 30, 'house': 31, 'women': 32, 'back': 33, 'white': 34, 'still': 35, 'make': 36, 'clinton': 37, '5': 38, 'world': 39, 'americans': 40, 'years': 41, 'way': 42, 'family': 43, 'black': 44, 'study': 45, 'gop': 46, 'best': 47, 'show': 48, 'bill': 49, 'american': 50, 'school': 51, 'watch': 52, '3': 53, 'police': 54, 'president': 55, 'really': 56, 'good': 57, 'cant': 58, 'know': 59, 'going': 60, 'video': 61, 'home': 62, 'death': 63, 'things': 64, '10': 65, 'finds': 66, 'last': 67, 'hillary': 68, 'love': 69, 'state': 70, 'nation': 71, 'may': 72, 'health': 73, 'parents': 74, 'say': 75, '2': 76, 'big': 77, 'work': 78, 'every': 79, 'need': 80, 'right': 81, 'dea

In [ ]:
import numpy as np

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)


<h1> The Model </h1>

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 12),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2  = 0.999, amsgrad=False)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7fb8dd382560>>

In [ ]:
num_epochs = 50
model.fit(training_padded, training_labels, epochs=num_epochs,
          validation_data=(testing_padded, testing_labels))

Epoch 1/50
688/688 [==============================] - 3s 3ms/step - loss: 0.8204 - accuracy: 0.5570 - val_loss: 0.7851 - val_accuracy: 0.5744
Epoch 2/50
688/688 [==============================] - 2s 3ms/step - loss: 0.7634 - accuracy: 0.5582 - val_loss: 0.7400 - val_accuracy: 0.5744
Epoch 3/50
688/688 [==============================] - 2s 3ms/step - loss: 0.7287 - accuracy: 0.5582 - val_loss: 0.7124 - val_accuracy: 0.5744
Epoch 4/50
688/688 [==============================] - 2s 4ms/step - loss: 0.7076 - accuracy: 0.5582 - val_loss: 0.6960 - val_accuracy: 0.5744
Epoch 5/50
688/688 [==============================] - 2s 3ms/step - loss: 0.6950 - accuracy: 0.5582 - val_loss: 0.6864 - val_accuracy: 0.5744
Epoch 6/50
688/688 [==============================] - 2s 3ms/step - loss: 0.6879 - accuracy: 0.5582 - val_loss: 0.6808 - val_accuracy: 0.5744
Epoch 7/50
688/688 [==============================] - 2s 3ms/step - loss: 0.6837 - accuracy: 0.5582 - val_loss: 0.6769 - val_accuracy: 0.5746
Epoch 

In [ ]:
sentence = ["dog flying int."]

In [ ]:
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[2357, 1, 1, 1]]


In [ ]:
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print("Result:", model.predict(padded)[0][0])

1/1 [==============================] - 0s 18ms/step
Result: 0.33162844
